In [1]:
import sys
import os
import gensim
sys.path.append("../..")  # adding Recommenders root path.
print(sys.path)

['/opt/caffe/python', '/opt/caffe2/build', '/data/home/wutao/src/Recommenders/notebooks/00_quick_start', '/data/anaconda/envs/reco_base/lib/python36.zip', '/data/anaconda/envs/reco_base/lib/python3.6', '/data/anaconda/envs/reco_base/lib/python3.6/lib-dynload', '', '/data/anaconda/envs/reco_base/lib/python3.6/site-packages', '/data/anaconda/envs/reco_base/lib/python3.6/site-packages/IPython/extensions', '/data/home/wutao/.ipython', '/data/anaconda/envs/reco_base/lib/python3.6/site-packages/azureml/_project/vendor', '../..']


In [2]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

import logging
import time

import numpy as np
import pandas as pd
#import papermill as pm
import gensim
import multiprocessing

from numpy import random
#from sklearn.cluster import KMeans

from reco_utils.dataset import movielens
from reco_utils.dataset.movielens import load_pandas_df
from reco_utils.dataset.python_splitters import python_stratified_split,python_chrono_split
#from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, 
    precision_at_k, recall_at_k, get_top_k_items)


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))


System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Pandas version: 0.24.2


In [3]:
data = load_pandas_df(size='100k', genres_col='genre', title_col='title', year_col='year')

# Convert the float precision to 32-bit in order to reduce memory consumption 
data['rating'] = data['rating'].astype(np.float32)
data[['userID','itemID']] = data[['userID','itemID']].astype(str)


100%|██████████| 4.81k/4.81k [00:00<00:00, 9.62kKB/s]


In [4]:
#train, test = python_stratified_split(data, ratio=0.75, col_user='userID', col_item='itemID', seed=42)
train, test = python_chrono_split(data, ratio=0.75, col_user='userID', col_item='itemID')
#train, validation, test = python_chrono_split(data, ratio=[0.7, 0.05, 0.25])
train_dict = dict(train.groupby('userID')['itemID'].apply(list))
test_dict = dict(test.groupby('userID')['itemID'].apply(list))
train_users_list = list(train_dict.keys()) 

In [5]:
train_distribution_list = [0] * len(train_users_list)

for i in range(len(train_users_list)):
    train_distribution_list[i] = len(train_dict[train_users_list[i]])

total_viewings = sum(train_distribution_list)
prob_train = [user_num_movies / total_viewings for user_num_movies in train_distribution_list]

In [20]:
ITER = 200000
WINDOW = 15+1
REP = 50
PROPORTION_USAGE = 0.80
ADD_UID = False
DIM=50

TOP_K = 10
TOP_K_MULTIPLIER = 1

TOPN_PERCLUSTER = 20
#MAX_CLUSTERS = 15

random.seed(2)

EXP_NAME = "chrono_200k_080_50dim_075_025"
datafile = EXP_NAME + ".data"
modelfile = EXP_NAME + ".model"

In [21]:
#NO NEED TO RUN IF data file is already available

with open(datafile,"w") as f:
    for i in range(ITER):
        if i / ITER < PROPORTION_USAGE:  # sample based on user's usage.  The more ratings a user has, the higher probability of being selected.
            user_idx = random.choice(len(prob_train),p=prob_train)
        else:  # go through user population sequentially
            user_idx = i % len(train_users_list)
            
        selected_userID = train_users_list[user_idx]
        selected_user_movies_list = train_dict[selected_userID]
        if len(selected_user_movies_list) >= WINDOW:
            window = WINDOW
        else:
            window = len(selected_user_movies_list)
        
        temp_movies_list = []
        uid_string = "u"+str(selected_userID)
        
        for j in range(REP):
# now switching to numpy.random            idxsample = random.sample(range(len(rand_user_movies_list)),window)
            idxsample = random.randint(len(selected_user_movies_list),size=window)
            temp_movies_list = temp_movies_list + [selected_user_movies_list[idx] for idx in idxsample]
            if ADD_UID:
                temp_movies_list.append(uid_string)
                
        f.write(' '.join(temp_movies_list)+'\n')
        

In [22]:
def read_seq():
    with open(datafile,"rt") as f:
        for i, line in enumerate(f):
            yield line.strip().split()

docs = list(read_seq())

In [23]:
start_time = time.time()
model = gensim.models.Word2Vec(docs, size = DIM, window = 5, min_count =2 , 
                               workers = multiprocessing.cpu_count(), sg = 1)
model.train(docs, total_examples= len(docs), epochs = 10, compute_loss = True)

(1492281758, 1598779500)

In [24]:
train_time = time.time() - start_time
print("Took {} seconds for training.".format(train_time))

Took 1113.841839313507 seconds for training.


In [25]:
model.save(modelfile)

In [26]:
model = gensim.models.Word2Vec.load(modelfile)

In [27]:
def remove_seen_and_uservec(predicted_list_w_scores, seen_list):
# predicted_list_w_scores is a list of lists,[[a,b]...] where a is ther user_id and b is the similairity score
    result_list = []
    for a, b in predicted_list_w_scores:
        if a not in seen_list and not a.startswith('u'):
            result_list.append([a, b])
    return result_list


def get_best_matches(userid):
    trained_wv_list = []
    for movie_id in train_dict[userid]:
        trained_wv_list.append(model.wv[movie_id])

    trained_wv_np = np.array(trained_wv_list)
    rec_list = []

#    for num_clusters in range(1,min(MAX_CLUSTERS,len(trained_wv_list))+1):
#     for num_clusters in range(len(trained_wv_list),len(trained_wv_list)+1): # don't do cluster
#         clustered_result = KMeans(n_clusters=num_clusters, random_state=0).fit(trained_wv_np)
#         centroids = clustered_result.cluster_centers_
    combined_rec_list = []
    for mov_vec in trained_wv_list:
        temp_recommended_list = model.wv.most_similar(positive = [mov_vec],topn=TOPN_PERCLUSTER)
        temp_rec_seen_removed_list = remove_seen_and_uservec(temp_recommended_list, train_dict[userid])
        combined_rec_list.extend(temp_rec_seen_removed_list)
    rec_list.extend(combined_rec_list)  # this is not to pick the set of recommended movies based on a fixed clustering, but rather taking all possible clusterings and find the highest similarity
    rec_list.sort(key = lambda element: element[1], reverse = True)  #rank based on similarity.  
    
# rec_list may have same movies with different scores.  Since it is sorted, in result_list we will only take the first score.

    rec_movieids_list = []
    rec_movies = 0
    rec_list_pointer = 0
    result_list = []
    while rec_movies < min(TOP_K * TOP_K_MULTIPLIER,len(rec_list)): # compute a larger list as input to the classifier; don't need it if not using a classifier
        if rec_list_pointer >= len(rec_list):
            break
        if rec_list[rec_list_pointer][0] not in rec_movieids_list:  #index 0 for movie id  
            rec_movieids_list.append(rec_list[rec_list_pointer][0])  #insert movie to rec_movieids_list
            element_list = [userid]
            element_list.extend(rec_list[rec_list_pointer])
            result_list.append(element_list)
            rec_movies = rec_movies + 1
        rec_list_pointer = rec_list_pointer + 1    
    return result_list

In [28]:
similarity_list = []

start_time = time.time()

for test_userid in test_dict:
    print(test_userid,end=",")
    similarity_list.extend(get_best_matches(test_userid))
    
prediction_time = time.time() - start_time

print("\nTook {} seconds for prediction".format(prediction_time))

1,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,135,136,137,138,139,14,140,141,142,143,144,145,146,147,148,149,15,150,151,152,153,154,155,156,157,158,159,16,160,161,162,163,164,165,166,167,168,169,17,170,171,172,173,174,175,176,177,178,179,18,180,181,182,183,184,185,186,187,188,189,19,190,191,192,193,194,195,196,197,198,199,2,20,200,201,202,203,204,205,206,207,208,209,21,210,211,212,213,214,215,216,217,218,219,22,220,221,222,223,224,225,226,227,228,229,23,230,231,232,233,234,235,236,237,238,239,24,240,241,242,243,244,245,246,247,248,249,25,250,251,252,253,254,255,256,257,258,259,26,260,261,262,263,264,265,266,267,268,269,27,270,271,272,273,274,275,276,277,278,279,28,280,281,282,283,284,285,286,287,288,289,29,290,291,292,293,294,295,296,297,298,299,3,30,300,301,302,303,304,305,306,307,308,309,31,310,311,312,313,314,315,316,317,318,319,32,320,321,322,323,324,325,326,327,328,329,33,33

In [29]:
predict_similarity_df = pd.DataFrame(similarity_list,columns=['userID','itemID','prediction'])
predict_similarity_df[['userID','itemID']] = predict_similarity_df[['userID','itemID']].astype(str)
predict_similarity_df.head()

,userID,itemID,prediction
0,1,588,0.961808
1,1,228,0.944447
2,1,436,0.925983
3,1,447,0.912425
4,1,29,0.904091


In [30]:
predict_similarity_file = EXP_NAME+'_DF.csv'
predict_similarity_df.to_csv(predict_similarity_file,index=False)

In [31]:
# p2 = pd.read_csv(predict_similarity_file)
# p2.head()

In [32]:
eval_map = map_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)
eval_ndcg = ndcg_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)
eval_precision = precision_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)
eval_recall = recall_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)

print("Word Embedding:")
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Word Embedding:
MAP:	0.045602
NDCG:	0.188750
Precision@K:	0.165960
Recall@K:	0.095186
